In [32]:
file_args_key = "live"
proof_number = 6
keynumber0816 = 4
kkkkkkkkkkkk = 0  
key_read_write = 'write'  
raise_Exception = False 
batch_sanpling_bur = True  

In [33]:
import torch

print(torch.__version__)

1.10.0+cu113


In [34]:
import torch
from torch import nn
import torch.nn.functional as F
from torchtext.legacy import data, datasets

from sklearn.model_selection import KFold
import copy
# from tqdm import tqdm
import numpy as np

# from model_nn import RNN, RNN_gradient,RNN50, RNN50_gradient
# from args_fedavg import args_parser_live, args_parser_shake
from model_nn_min import RNN, RNN_gradient,RNN50, RNN50_gradient
from args_fvg import args_parser_live, args_parser_shake

from nltk.tag import pos_tag
import re
import time


In [35]:
out_log = r"/mnt/workspace/fedavg_ceshi/out_log"
out_pth = r"/mnt/workspace/fedavg_ceshi/out_pth"
write_read_txt = r"/mnt/workspace/fedavg_ceshi/write_read_txt"

if file_args_key == "shake":
    args = args_parser_shake()
elif file_args_key == "live":
    args = args_parser_live()

String = time.strftime("%m_%d_%Y_%H_%M_%S",time.localtime())
namestr = str(String)

file_write_log_name = out_log + "/" + "log_" + file_args_key + '_'  + str(proof_number) \
    + '_'+ str(keynumber0816) + '_'+ str(kkkkkkkkkkkk) + '_' \
        + str(args.B) + '_'+ str(args.C) + '_' + key_read_write + '_'+ namestr + "_log.txt"

print(file_write_log_name)

file_write_log = open(file_write_log_name, "w", encoding='utf-8')

save_name_pth = out_pth + "/" +  file_args_key + '_'  + str(proof_number) \
    + '_'+ str(keynumber0816) + '_'+ str(kkkkkkkkkkkk) + '_' \
        + str(args.B) + '_'+ str(args.C) + '_' + key_read_write + '_'+ namestr +'_dict.pth'

print(save_name_pth)

writecon = str(save_name_pth) + "\n"
file_write_log.write(writecon)

/mnt/workspace/fedavg_ceshi/out_log/log_live_6_4_0_30_0.5_write_08_27_2023_11_08_42_log.txt
/mnt/workspace/fedavg_ceshi/out_pth/live_6_4_0_30_0.5_write_08_27_2023_11_08_42_dict.pth


89

In [36]:
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

if file_args_key == "shake":
    # train_data.examples = train_data.examples[:len(train_data.examples)//500]
    # test_data.examples = test_data.examples[:len(test_data.examples)//500]

    train_data.examples = train_data.examples[:len(train_data.examples)//125]
    test_data.examples = test_data.examples[:len(test_data.examples)//125]

len_train_data = len(train_data)
len_test_data = len(test_data)

print('len of train data:', len(train_data))
print('len of test data:', len(test_data))
print(test_data.examples[1].text) 
print(test_data.examples[1].label)


len of train data: 25000
len of test data: 25000
['I', 'was', 'five', 'when', 'the', 'show', 'made', 'its', 'debut', 'in', '1958', 'and', 'at', 'a', 'later', 'point', ',', 'was', 'a', 'regular', 'viewer', '.', 'I', 'remember', 'that', 'I', 'really', 'enjoyed', 'the', 'show', ',', 'along', 'with', '"', 'Leave', 'It', 'To', 'Beaver', '"', ',', '"', 'My', 'Three', 'Sons', '"', ',', '"', 'Ozzie', 'and', 'Harriet', '"', ',', '"', 'Dick', 'Van', 'Dyke', '"', ',', 'reruns', 'of', '"', 'I', 'Love', 'Lucy', '"', ',', '"', 'The', 'Real', 'McCoys', '"', ',', 'etc', '.', 'I', 'am', 'now', 'enjoying', 'the', 'first', 'season', 'of', '"', 'Donna', 'Reed', '"', 'on', 'DVD', 'and', 'have', 'watched', 'the', 'first', 'two', 'episodes', '.', 'Donna', 'Stone', 'is', 'shown', 'to', 'be', 'an', 'intelligent', ',', 'well', '-', 'mannered', ',', 'problem', '-', 'solving', ',', 'serene', ',', 'stay', '-', 'at', '-', 'home', 'mom', ',', 'similar', 'to', 'June', 'Cleaver', 'and', 'in', 'contrast', 'to', 'Lucy',

In [37]:
# import nltk
# nltk.download('averaged_perceptron_tagger')

a = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a

device(type='cuda')

In [38]:
TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.100d')
LABEL.build_vocab(train_data)
vocab_size =  len(TEXT.vocab)
pretrained_embedding = TEXT.vocab.vectors

print(vocab_size, pretrained_embedding.shape)

10002 torch.Size([10002, 100])


In [39]:
def RFFM(i, x):
    torch.manual_seed(i)
    input_dim = 100
    output_dim = 50
    sigma = 1.0
    omega = torch.randn(output_dim, input_dim, device=args.device) * sigma
    b = torch.rand(output_dim, device=args.device) * 2 * np.pi
    x = x.view(-1, input_dim).to(args.device)
    z = torch.cos(torch.mm(x, omega.t()) + b)
    Z_Out =  torch.sqrt(2.0 / torch.tensor(output_dim)) * z
    return Z_Out.to('cpu')

In [40]:
new_vectors = []
for i, word in enumerate(TEXT.vocab.itos):
    old_vector = pretrained_embedding[i]
    new_vector = RFFM(i, old_vector)
    new_vectors.append(new_vector)

new_pretrained_embedding = torch.stack(new_vectors, dim=0)
new_pretrained_embedding = new_pretrained_embedding.reshape(-1,50)
print(new_pretrained_embedding.shape)

torch.Size([10002, 50])


In [41]:
def word_vector_50(word):
    word_index = TEXT.vocab.stoi[word]
    word_vector = new_pretrained_embedding[word_index]
    return word_vector

def proof1(a, b, epcluo):
    a = a.to(torch.float)
    b = b.to(torch.float)
    dot_product = torch.dot(a, b)
    norm_a = torch.norm(a, p=2)
    norm_b = torch.norm(b, p=2)
    similarity = dot_product / (norm_a * norm_b)
    return torch.exp(similarity* epcluo / 2.0 *(-1))

def proof1_change(word_list,  proper_nouns, filtered, proper_nouns_pos):
    if len(proper_nouns) == 0 or len(filtered) == 0 :  
        return word_list
    epcluo = args.epcluo
    gam = []

    xlist = [word_vector_50(word) for word in proper_nouns]
    ylist = [word_vector_50(word) for word in filtered]
    
    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof1(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)

    gam_tensor = torch.tensor(gam)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True)  
    result_prob = torch.div(gam_tensor, row_sums) 
    sample = torch.multinomial(result_prob, 1)
    list_result = sample.squeeze().tolist()
    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]
    rresul = [filtered[i] for i in list_result]
    iiindex = 0

    for index, word in enumerate(proper_nouns_pos):
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            newword = rresul[iiindex] 
            word_list[index] = newword
            iiindex += 1   
    
    return word_list

In [42]:
def word_vector_100(word):
    word_index = TEXT.vocab.stoi[word]
    word_vector = TEXT.vocab.vectors[word_index]
    return word_vector

def proof2(x, y, epcluo):
    K = torch.exp(torch.norm(x-y, p=2).pow(2) / 2.0 *(-1) )
    result = K * epcluo / 2.0 *(-1) 
    e14 = torch.exp(result)
    return e14

def proof2_change(word_list,  proper_nouns, filtered, proper_nouns_pos):
    if len(proper_nouns) == 0 or len(filtered) == 0 :  
        return word_list
    epcluo = args.epcluo
    gam = []
    xlist = [word_vector_100(word) for word in proper_nouns]
    ylist = [word_vector_100(word) for word in filtered]
    
    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof2(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)

    gam_tensor = torch.tensor(gam)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True) 
    result_prob = torch.div(gam_tensor, row_sums)  
    sample = torch.multinomial(result_prob, 1)
    list_result = sample.squeeze().tolist()

    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]
    rresul = [filtered[i] for i in list_result]
    iiindex = 0

    for index, word in enumerate(proper_nouns_pos):
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            newword = rresul[iiindex]
            word_list[index] = newword
            iiindex += 1

    return word_list

In [43]:
def proof3(zx, zy, epcluo): 
    result = (torch.dot(zx, zy)) * epcluo / 4.0 *(-1) 
    e14 = torch.exp(result)
    return e14

def proof3_change(word_list,  proper_nouns, filtered, proper_nouns_pos):
    if len(proper_nouns) == 0 or len(filtered) == 0 : 
        return word_list
    epcluo = args.epcluo
    gam = []
    xlist = [word_vector_50(word) for word in proper_nouns]
    ylist = [word_vector_50(word) for word in filtered]
    
    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof3(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)
    gam_tensor = torch.tensor(gam)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True) 
    result_prob = torch.div(gam_tensor, row_sums)  
    sample = torch.multinomial(result_prob, 1)
    list_result = sample.squeeze().tolist()
    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]
    rresul = [filtered[i] for i in list_result]
    iiindex = 0
    for index, word in enumerate(proper_nouns_pos):
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            newword = rresul[iiindex]
            word_list[index] = newword
            iiindex += 1

    return word_list

In [44]:
def proof6(zx, zy, epcluo):
    result = (torch.dot(zx, zy)) * epcluo / 4.0 *(-1) 
    e14 = torch.exp(result)
    return e14

def proof6_change(word_list,  proper_nouns, filtered, proper_nouns_pos, filtered_y, proper_nouns_x ):
    if len(proper_nouns) == 0 or len(filtered) == 0 :  
        return word_list
    
    epcluo = args.epcluo
    gam = []
    xlist = [word_vector_50(word) for word in proper_nouns]
    ylist = [word_vector_50(word) for word in filtered]
    ylist_f = [word_vector_50(word) for word in filtered_y]
    xlist_f = [word_vector_50(word) for word in proper_nouns_x]
    
    for x in xlist:
        x_list = []
        for y in ylist:
            res = proof6(x, y, epcluo)
            x_list.append(res.item())
        gam.append(x_list)

    gam_tensor = torch.tensor(gam)
    row_sums = torch.sum(gam_tensor, dim=1, keepdim=True)  
    result_prob = torch.div(gam_tensor, row_sums)  
    sample = torch.multinomial(result_prob, 1)
    tensor = sample.clone()
    list_result = tensor.squeeze().tolist()
    try:
        list_result = list(list_result)
    except:
        list_result = [list_result]
    rresul = [filtered[i] for i in list_result]
    pr = args.pr
    gam_pr = []

    for y in ylist_f:
        x_list_new = []
        for x in xlist_f:
            res = proof6(y, x, epcluo)
            x_list_new.append(res.item())
        gam_pr.append(x_list_new)

    gam_pr_tensor = torch.tensor(gam_pr)
    row_pr_sums = torch.sum(gam_pr_tensor, dim=1, keepdim=True)  
    result_prob = pr * torch.div(gam_pr_tensor, row_pr_sums)  
    column_to_add = torch.full((result_prob.shape[0], 1), pr)
    tensor_with_column = torch.cat((result_prob, column_to_add), dim=1)
    sample_pr = torch.multinomial(tensor_with_column, 1)
    tensor_pr = sample_pr.clone()
    list_result_pr = tensor_pr.squeeze().tolist()

    try:
        list_result_pr = list(list_result_pr)
    except:
        list_result_pr = [list_result_pr]

    rresuly = []
    xylong = len(tensor_with_column[0,:]) - 1

    for index, yword in enumerate(list_result_pr):
        if yword == xylong :
            rresuly.append(filtered_y[index])
        else:
            rresuly.append(proper_nouns_x[yword])
    iiindex = 0
    iiindey = 0
    
    for index, word in enumerate(proper_nouns_pos):
        if (word[0] in proper_nouns) and (word[1].startswith('NNP') or word[1].startswith('NNPS') ):
            newword = rresul[iiindex]
            word_list[index] = newword
            iiindex += 1

        elif (word[0] in filtered_y) and not (word[1].startswith('NNP') or word[1].startswith('NNPS')) :  
            newword = rresuly[iiindey]
            word_list[index] = newword
            iiindey += 1

    return word_list

In [45]:
def filter_proper_nouns(words):
    tagged_words = pos_tag(words)
    proper_nouns = []
    filtered_y = []
    proper_nouns_pos = []

    for word, pos in tagged_words:
        if pos.startswith('NNP') or pos.startswith('NNPS'):
            proper_nouns.append(word)
        else:
            filtered_y.append(word)
        proper_nouns_pos.append((word,pos))
    return proper_nouns, proper_nouns_pos, filtered_y


def zxy_split(word_list, proof_number):
    proper_nouns, proper_nouns_pos, filtered_y = filter_proper_nouns(word_list)
    filtered = list(set(word_list) - set(proper_nouns))

    if  proof_number == 1:
        new_word_list = proof1_change(word_list,  proper_nouns, filtered, proper_nouns_pos)

    elif proof_number == 2:
        new_word_list = proof2_change(word_list,  proper_nouns, filtered, proper_nouns_pos)

    elif proof_number == 3:
        new_word_list = proof3_change(word_list,  proper_nouns, filtered, proper_nouns_pos)

    elif proof_number == 6:
        proper_nouns_x = list(set(proper_nouns))
        new_word_list = proof6_change(word_list,  proper_nouns, filtered, proper_nouns_pos, filtered_y, proper_nouns_x)
    elif proof_number == 0:    
        new_word_list = word_list
    else:
        pass
    return new_word_list


In [46]:
def geshj_tra_tes(data):
    for index in range(len(data)):
        if ((index+1) % 1000 == 0):
            print(index)
        atexta = data.examples[index].text
        filtered_aaa0 = [word for word in atexta if re.match(r'^[A-Za-z.,!?0-9]+$', word)]
        word_listaaa = zxy_split(filtered_aaa0, proof_number)
        data.examples[index].text = word_listaaa


In [47]:
def chulia(astring):
    astring = astring.strip('\n').split('#')
    astring = [i for i in astring if len(i)>0]
    return astring

def train_test_read(tr_te, data):
    for index in range(len(tr_te)): 
        word_listaaa = data[index]
        tr_te.examples[index].text = word_listaaa
        if (file_args_key == "live") and (index % 1000 == 0):
            print(index)
        if (file_args_key == "shake") and (index % args.B == 0):
            print(index)

def read_or_write(key, proof_number):
    # file_args_key = "live"
    # file_args_key = "shake"
    if proof_number == 1:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof1_norm.txt'
            file_name_test  = 'test_data_change_result_proof1_norm.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof1_50_norm.txt'
            file_name_test = 'test__proof1_50_norm.txt'

    if proof_number == 2:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof2_norm.txt'
            file_name_test = 'test_data_change_result_proof2_norm.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof2_50_norm.txt'
            file_name_test = 'test__proof2_50_norm.txt'

    if proof_number == 3:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof3_norm.txt'
            file_name_test = 'test_data_change_result_proof3_norm.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof3_50_norm.txt'
            file_name_test = 'test__proof3_50_norm.txt'

    if proof_number == 6:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof6_norm.txt'
            file_name_test  = 'test_data_change_result_proof6_norm.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof6_50_norm.txt'
            file_name_test = 'test__proof6_50_norm.txt'
    
    if proof_number == 0:
        if file_args_key == "live":
            file_name_train = 'train_data_change_result_proof0_norm.txt'
            file_name_test  = 'test_data_change_result_proof0_norm.txt'
        elif file_args_key == "shake":
            file_name_train = 'train_proof0_50_norm.txt'
            file_name_test = 'test__proof0_50_norm.txt'

    file_name_train = write_read_txt + "/" + file_name_train
    file_name_test = write_read_txt + "/" + file_name_test

    if key=='read':
        fead_train = open(file_name_train, 'r', encoding='utf-8').readlines()
        fead_test = open(file_name_test, 'r', encoding='utf-8').readlines()
        train_test_read(train_data, fead_train)
        train_test_read(test_data, fead_test)

    elif key=='write':
        geshj_tra_tes(train_data)
        file_write(file_name_train, len_train_data, train_data)
        geshj_tra_tes(test_data)
        file_write(file_name_test, len_test_data, test_data)

    else:
        pass

def list_to_string(alist):
    string = ''
    for i in alist:
        string += i + '#'
    return string

def file_write(filename, len_data, data):
    f = open(filename, 'w', encoding = 'utf-8')
    for index in range(len_data):
        strings = list_to_string(data.examples[index].text)
        f.write(strings + '\n')
    f.close()


In [48]:
read_or_write(key_read_write, proof_number)
if raise_Exception and (key_read_write == 'write') :
    raise Exception("Terminated at this point")


999
1999
2999
3999
4999
5999
6999
7999
8999
9999
10999
11999
12999
13999
14999
15999
16999
17999
18999
19999
20999
21999
22999
23999
24999
999
1999
2999
3999
4999
5999
6999
7999
8999
9999
10999
11999
12999
13999
14999
15999
16999
17999
18999
19999
20999
21999
22999
23999
24999


In [49]:
train_indices = list(range(len(train_data.examples)))

k = args.K 
random_seed = 42  
kfold = KFold(n_splits=k, shuffle=True, random_state=random_seed)

train_subsets = []
for _, train_idx in kfold.split(train_indices):
    train_subset = data.Dataset([train_data.examples[i] for i in train_idx], fields=[('text', TEXT), ('label', LABEL)])
    train_subsets.append(train_subset)

In [50]:
test_indices = list(range(len(test_data.examples)))

random_seed = 24  
kfold_te = KFold(n_splits=k, shuffle=True, random_state=random_seed)

test_subsets = []
for _, test_idx in kfold_te.split(test_indices):
    test_subset = data.Dataset([test_data.examples[i] for i in test_idx], fields=[('text', TEXT), ('label', LABEL)])
    test_subsets.append(test_subset)

In [51]:
def iterator(tri_data, tes_data):
    
    train_iterator, test_iterator = data.BucketIterator.splits(
        (tri_data, tes_data),
        batch_size = args.B,
        sort_key = lambda x: len(x.text),
        sort_within_batch=True,
        device=torch.device("cpu") 
    )

    return train_iterator, test_iterator

In [52]:
train_all_sub = []
testa_all_sub = []

for index in range(len(train_subsets)) :
    data_i, data_j = train_subsets[index ], test_subsets[index ]
    data_i_reu, data_j_reu = iterator(data_i, data_j)
    train_all_sub.append(data_i_reu)
    testa_all_sub.append(data_j_reu)

In [53]:
loss_function = nn.BCEWithLogitsLoss().to(args.device) 
loss_function_gradient = nn.BCEWithLogitsLoss().to(args.device) 

def get_val_loss(model, Vall):  
    model = model.to(args.device)
    model.eval()
    val_loss = []
    with torch.no_grad():
        for seqq in Vall:
            text = seqq.text.to(args.device)
            target = seqq.label.to(args.device)
            y_preda = model(text).squeeze(1)
            loss = loss_function(y_preda, target)
            val_loss.append(loss.item())
    model = model.to('cpu')        
    loss_v = np.array(val_loss).mean()
    return loss_v 

def binary_acc(preds, y):
    preds = preds.to('cpu')
    y = y.to('cpu')
    preds = torch.round(torch.sigmoid(preds))
    correct = torch.eq(preds, y).float()
    acc = correct.sum() / len(correct)
    return acc

def test(args, model, k):
    model.eval()
    Val = testa_all_sub[k] 
    avg_acc = []

    with torch.no_grad():
        for seq in Val:
            text = seq.text.to(args.device)
            y_pred = model(text).squeeze(1) 
            y_pred = y_pred.to('cpu')
            target = seq.label.to('cpu')
            acc = binary_acc(y_pred, target).item()
            avg_acc.append(acc)
    avg_acc = np.mean(avg_acc)
    
    print("客户端：", k, ' avg acc', avg_acc)

    wstring = f"客户端：{k} avg acc: {avg_acc}"
    writecon = str(wstring) + "\n"
    file_write_log.write(writecon)

In [54]:
def selectsampling(x):
    custom_min = 0.0001
    custom_max = 0.9999
    scaled_data = (x.max() - x) / (x.max() - x.min()) * (custom_max - custom_min) + custom_min
    probs = F.softmax(scaled_data, dim=0)
    return probs    

def random_selectsampling_batch(Len):
    return torch.full([Len], 1.0 / Len)


In [55]:
def yangbenshaixuan_batch(Dtr, norms, sampling_rate, keynumber0816, Len):
    if keynumber0816 == 1:
        pi = selectsampling(norms)
    elif keynumber0816 == 4:
        pi = random_selectsampling_batch(Len)
    else:
        pass

    num_samples_num = int(sampling_rate * len(pi))
    try:
        samples = torch.multinomial(pi, num_samples= num_samples_num, replacement=False)
    except:
        print("---except---")
        pi = random_selectsampling_batch(Len)
        samples = torch.multinomial(pi, num_samples= num_samples_num, replacement=False)

    new_sample_batch = []
    samples = samples.tolist()
    iiindex_list = []
    for iiindex, batch in enumerate(Dtr):
        if iiindex in samples:
            new_sample_batch.append(batch)
            iiindex_list.append(iiindex)

    wstring = f"minigrid sampling batch:  {iiindex_list} "
    writecon = str(wstring) + "\n"
    file_write_log.write(writecon)
    return new_sample_batch

In [56]:
def real_select_batch(Dtr, gradient_model, optimizer_gradient):
    result = []
    for ttime, x_data in enumerate(Dtr):
        text = x_data.text.to(args.device)
        target = x_data.label.to(args.device)
        y_pred = gradient_model(text).squeeze(1)
        loss = loss_function_gradient(y_pred, target)
        loss.backward(retain_graph=True)
        grad_vars = []
        for param in gradient_model.parameters():
            if param.grad is not None:
                grad_vars.append(param.grad.cpu().detach().clone())
            else:
                pass

        for param in gradient_model.parameters():
            if param.grad is not None:
                a_grad = param.grad.detach_()
                param.grad.zero_()
        c = torch.cat([param.view(-1) for param in grad_vars]).unsqueeze(0).to("cpu")
        result.append(c)  

    result = torch.cat(result, dim=0)
    myresult = result.to('cpu')
    norms = torch.norm(myresult, dim=1)
    return  norms  


In [57]:
def train_batch(args, model, k, gradient_model):  
    Dtr = train_all_sub[k]   
    Val = testa_all_sub[k]
    
    count = 0
    for _ in Dtr:
        count += 1

    if args.optimizer == 'adam': 
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr,
                                     weight_decay=args.weight_decay)  
        optimizer_gradient = torch.optim.Adam(gradient_model.parameters(), lr=args.lr,
                                     weight_decay=args.weight_decay)  
    else:
        optimizer = torch.optim.SGD(model.parameters(), lr=args.lr,
                                    momentum=0.9, weight_decay=args.weight_decay)
        optimizer_gradient = torch.optim.SGD(gradient_model.parameters(), lr=args.lr,
                                    momentum=0.9, weight_decay=args.weight_decay)
    min_epochs = 1
    best_model = None
    min_val_loss = 10000.0

    for epoch in range(args.E):  
        train_loss = [] 
        if keynumber0816 == 1:
            gradient_model.load_state_dict(model.state_dict())
            gradient_model.train()
            gradient_model = gradient_model.to(args.device)
            norms = real_select_batch(Dtr, gradient_model, optimizer_gradient)
            gradient_model = gradient_model.to('cpu')
            batch_list = yangbenshaixuan_batch(Dtr, norms , args.C, keynumber0816, count) #GGG

        elif keynumber0816 == 4:
            batch_list = yangbenshaixuan_batch(Dtr, None, args.batch_sample_rate, keynumber0816, count)
        
        model = model.to(args.device)
        
        for _ in range(2):  
            for  X_train in  batch_list: 
                text = X_train.text.to(args.device)  
                target = X_train.label.to(args.device)
                model.train()
                y_pred = model(text).squeeze(1)
                loss = loss_function(y_pred, target)
                train_loss.append(loss.item())
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

        model = model.to('cpu')    
        gradient_model.load_state_dict(model.state_dict())
        val_loss = get_val_loss(model, Val)
        if (file_args_key == "shake") and ((epoch+1) % 1 == 0) :
            wstring = f"epoch {epoch} 训练 train_loss {np.array(train_loss).mean()} 验证 val_loss {val_loss}"
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)

        if (file_args_key == "live") and ((epoch+1) % 1 == 0):
            wstring = f"epoch {epoch} 训练 train_loss {np.array(train_loss).mean()} 验证 val_loss {val_loss}"
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)

        if (epoch + 1 >= min_epochs) and (val_loss <= min_val_loss) : 
            min_val_loss = val_loss
        
    model = model.to('cpu')
    avg_val_lossa = np.array(train_loss).mean()
    wstring = f"model {k} val_loss: {avg_val_lossa} "
    writecon = str(wstring) + "\n"
    file_write_log.write(writecon)
    wstring = f"训练 best min train_loss {min_val_loss} "
    writecon = str(wstring) + "\n"
    file_write_log.write(writecon)
    print('model {}, avg_loss {:.8f},  min_loss {:.8f}, val loss {:.8f} '.format(k, avg_val_lossa,min_val_loss, val_loss ))

    return model


In [58]:
class FedAvg:

    def __init__(self, args):

        self.args = args 
        self.clients = args.clients  
        
        if (proof_number == 1) or (proof_number == 3) or (proof_number == 6):
            self.nn = RNN50(vocab_size, args, name='server').to('cpu')   # 
            self.nn.embedding.weight.data.copy_(new_pretrained_embedding)
            
            self.nn_gradient = RNN50_gradient(vocab_size, args, name='nn_gradient').to('cpu')  
            self.nn_gradient.embedding.weight.data.copy_(new_pretrained_embedding)
        
        elif (proof_number == 0) or (proof_number == 2) :
            self.nn = RNN(vocab_size, args, name='server').to('cpu')
            self.nn.embedding.weight.data.copy_(pretrained_embedding)

            self.nn_gradient = RNN_gradient(vocab_size, args, name='nn_gradient').to('cpu')
            self.nn_gradient.embedding.weight.data.copy_(pretrained_embedding)

        self.nns = []
        self.m = 0
        
        for i in range(self.args.K):   
            temp = copy.deepcopy(self.nn)
            temp.name = self.clients[i]  
            self.nns.append(temp)  

    def server(self):
        for t in range(self.args.r):  
            print('round：--------', t + 1, '------------------------')
            wstring = f"round：-------- {t + 1} ------------------------"
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)
            m = max(int(self.args.C * self.args.K), 1)
            index = torch.randperm(self.args.K)[:m].tolist()
            self.m = m
            print('选取客户端:',index)

            wstring = f"选取客户端: {index} "
            writecon = str(wstring) + "\n"
            file_write_log.write(writecon)

            self.dispatch(index) 
            self.client_update(index)  
            self.aggregation(index)  

        torch.save(self.nn.state_dict(), save_name_pth)
        print(save_name_pth, 'save')
        return self.nn

    def dispatch(self, index): 
        self.nn = self.nn.to('cpu')
        for j in index:
            self.nns[j] = self.nns[j].to('cpu')
            for old_params, new_params in zip(self.nns[j].parameters(), self.nn.parameters()):
                old_params.data = new_params.data.clone()
            self.nns[j] = self.nns[j].to('cpu')
        self.nn = self.nn.to('cpu')

    def client_update(self, index): 
        if batch_sanpling_bur:
            for k in index: 
                wstring = f"训练本地模型： {k} "
                writecon = str(wstring) + "\n"
                file_write_log.write(writecon)
                self.nns[k] = self.nns[k].to('cpu')
                self.nns[k] = train_batch(self.args, self.nns[k], k, self.nn_gradient)  


    def aggregation(self, index):  
        s = 0

        params = {}
        self.nns[0] = self.nns[0].to('cpu')
        for k, v in self.nns[0].named_parameters(): 
            params[k] = torch.zeros_like(v.data)

        for j in index:
            self.nns[j] = self.nns[j].to('cpu')
            for k, v in self.nns[j].named_parameters():  
                params[k] += v.data * (1.0 / self.m ) 

        for k, v in self.nn.named_parameters():
            v.data = params[k].data.clone() 

    def global_test(self, args): 
        model = self.nn.to(args.device) 
        c = args.clients 
        for k, client in enumerate(c):
            test(self.args, model, k)


In [59]:
fed = FedAvg(args)
fed.server()

round：-------- 1 ------------------------
选取客户端: [16, 14, 7, 15, 17, 18, 12, 3, 6, 9]
model 16, avg_loss 0.03561878,  min_loss 0.69264111, val loss 1.81281111 
model 14, avg_loss 0.41568672,  min_loss 0.68919823, val loss 0.79486532 
model 7, avg_loss 0.07939017,  min_loss 0.69314045, val loss 1.41536689 
model 15, avg_loss 0.06293663,  min_loss 0.69053440, val loss 1.60556977 
model 17, avg_loss 0.05497739,  min_loss 0.69287919, val loss 1.67179835 
model 18, avg_loss 0.55379901,  min_loss 0.68827076, val loss 0.83264735 
model 12, avg_loss 0.02339429,  min_loss 0.69179591, val loss 2.18660032 
model 3, avg_loss 0.03766901,  min_loss 0.69287143, val loss 1.95140138 
model 6, avg_loss 0.06569693,  min_loss 0.68461872, val loss 1.63676541 
model 9, avg_loss 0.03744979,  min_loss 0.69397536, val loss 1.32547286 
round：-------- 2 ------------------------
选取客户端: [18, 7, 12, 15, 9, 11, 0, 2, 8, 10]
model 18, avg_loss 0.68558748,  min_loss 0.68779528, val loss 0.68944121 
model 7, avg_loss 0

RNN50(
  (embedding): Embedding(10002, 50)
  (rnn): LSTM(50, 256, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [60]:
fed.global_test(args)

客户端： 0  avg acc 0.7043650817303431
客户端： 1  avg acc 0.678968262104761
客户端： 2  avg acc 0.6777777806633994
客户端： 3  avg acc 0.6932539734102431
客户端： 4  avg acc 0.7103174641018822
客户端： 5  avg acc 0.6944444463366554
客户端： 6  avg acc 0.7222222245874859
客户端： 7  avg acc 0.6904761954432442
客户端： 8  avg acc 0.7023809552192688
客户端： 9  avg acc 0.7134920642489478
客户端： 10  avg acc 0.6960317464101882
客户端： 11  avg acc 0.6821428608326685
客户端： 12  avg acc 0.6888888875643412
客户端： 13  avg acc 0.6956349235205423
客户端： 14  avg acc 0.6849206373805091
客户端： 15  avg acc 0.7000000029802322
客户端： 16  avg acc 0.6976190535795121
客户端： 17  avg acc 0.6936507948807308
客户端： 18  avg acc 0.7079365111532665
客户端： 19  avg acc 0.6980158715021043


In [61]:
file_write_log.close()